In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('drive/MyDrive/식물을 부탁해/kjw')

In [ ]:
import json
import cv2
import random
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
# 음...이건 그냥 코드 돌릴때 일정하게 나올라고 설정한 거임.
# 1. 파이썬 내장 random 시드 고정
random.seed(42)

# 2. 넘파이 시드 고정
np.random.seed(42)

# 3. TensorFlow 시드 고정
tf.random.set_seed(42)

# 4. 환경 변수 (특히 GPU 연산의 비결정성 제어)
os.environ["PYTHONHASHSEED"] = "42"
os.environ["TF_DETERMINISTIC_OPS"] = "1"  # TF 연산을 결정적으로

# 식물종 명시(필수!!!!)

In [ ]:
plant_name = "스파티필럼"

In [ ]:

# (1) 드라이브 → Colab으로 복사
!cp "/content/drive/MyDrive/식물을 부탁해/kjw/dataset(detection)/스파티필럼_dataset_detection.zip" "/content/"

# (2) Colab에서 압축 해제
!unzip -q "/content/스파티필럼_dataset_detection.zip" -d "/content/스파티필럼_dataset_detection"

# (3) 압축 파일 삭제 (용량 절약)
!rm "/content/스파티필럼_dataset_detection.zip"

In [ ]:
# # ???_dataset_classification
# # (1) 드라이브 → Colab으로 복사
# !cp "/content/drive/MyDrive/AI+X고급/식물을 부탁해/kjw/dataset(classification)/선인장_dataset_classification.zip" "/content/"

# # (2) Colab에서 압축 해제
# !unzip -q "/content/선인장_dataset_classification.zip" -d "/content/선인장_dataset_classification"

# # (3) 압축 파일 삭제 (용량 절약)
# !rm "/content/선인장_dataset_classification.zip"

## 1. 객체 탐지(YOLO11n) 모델링

In [ ]:
# YOLOv11 설치
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.8 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
from ultralytics import YOLO
import os

data_yaml  = f"/content/{plant_name}_dataset_detection/data.yaml"
project    = f"yolo_log/{plant_name}_yolo/detect"

AUG = dict(
    hsv_h=0.015, hsv_s=0.60, hsv_v=0.60,
    degrees=5.0, translate=0.08, scale=0.30, shear=2.0,
    mosaic=1.0, mixup=0.15, copy_paste=0.15, close_mosaic=20,
    fliplr=0.5, flipud=0.0,
)

# --- Stage 1: 960 / 15 epochs ---
print("=== Stage 1: 960 / 15 epochs ===")
model = YOLO("yolo11n.pt")
stage1_name = "stage1_960"

results_stage1 = model.train(
    data=data_yaml,
    epochs=15,
    imgsz=960,
    batch=8,          # 정수로 지정 (OOM 시 4 또는 2로 낮추기)
    workers=1,
    cache=True,
    device=0,
    project=project,
    name=stage1_name,
    exist_ok=True,
    **AUG
)

# --- Stage 2: 1280 / +10 epochs (total 25) ---
print("=== Stage 2: 1280 / +10 epochs (total 25) ===")
best_stage1 = os.path.join(project, stage1_name, "weights", "best.pt")
assert os.path.exists(best_stage1), f"Stage1 best.pt not found: {best_stage1}"

model1280 = YOLO(best_stage1)
stage2_name = "stage2_1280_ft"

results_stage2 = model1280.train(
    data=data_yaml,
    epochs=10,
    imgsz=1280,
    batch=4,          # 1280은 메모리↑ → 더 낮게 (OOM 시 2 또는 1)
    workers=1,
    cache=True,
    device=0,
    project=project,
    name=stage2_name,
    exist_ok=True,
    **AUG
)

print(f"Stage1(best): {best_stage1}")
print(f"Stage2 runs : {os.path.join(project, stage2_name)}")
print("→ 최종 배포 후보 가중치: stage2_1280_ft/weights/best.pt (또는 last.pt)")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
=== Stage 1: 960 / 15 epochs ===
Ultralytics 8.3.231 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=20, cls=0.5, compile=False, conf=None, copy_paste=0.15, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/스파티필럼_dataset_detection/data.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.6, hsv_v=0.

In [ ]:
# -----------------------------
# 최종 모델 검증(Validation)
#  - stage2(1280) best.pt 기준
# -----------------------------

best_stage2 = os.path.join(project, stage2_name, "weights", "best.pt")
assert os.path.exists(best_stage2), f"Stage2 best.pt not found: {best_stage2}"

val_model = YOLO(best_stage2)

metrics = val_model.val(
    data=data_yaml,          # 동일 data.yaml
    imgsz=1280,              # 학습 단계2와 동일 해상도
    batch=8,                 # ★ 정수로 지정 (OOM나면 4 또는 2로)
    device=0,
    project=project,
    name="val_stage2_1280",  # 결과 저장 폴더
    exist_ok=True,
    plots=True,              # PR/Confusion 등 그래프 저장
    save_json=True           # COCO 포맷 메트릭 JSON 저장
)
print(metrics)


Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3320.1±760.1 MB/s, size: 1830.0 KB)
val: Scanning /content/디펜바키아_dataset_detection/val/labels.cache... 1800 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1800/1800 2.9Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 1% ──────────── 2/225 1.7s/it 1.8s<6:22


KeyboardInterrupt: 

In [ ]:
# -----------------------------
# Test 세트 예측 (Prediction)
#  - stage2(1280) best.pt 기준
#  - 작은 잎까지 최대한 많이 탐지하도록 리콜 중심 파라미터
# -----------------------------

best_stage2 = os.path.join(project, stage2_name, "weights", "best.pt")
assert os.path.exists(best_stage2), f"Stage2 best.pt not found: {best_stage2}"

test_model = YOLO(best_stage2)

pred = test_model.predict(
    source=f"/content/{plant_name}_dataset_detection/test/images",  # 테스트 이미지 폴더
    data=data_yaml,
    save=True,                  # 탐지 결과 이미지 저장
    imgsz=1280,                 # 학습(Stage2)와 동일 해상도
    conf=0.18,                  # ↓ 낮춰서 작은 잎 후보 더 수집(0.18~0.25 범위 조정 가능)
    iou=0.70,                   # ↑ 겹친 잎 박스 유지 (NMS 덜 지우게)
    max_det=300,                # 이미지당 박스 상한 늘리기
    project=project,
    name="predict_stage2_1280", # 결과 폴더명
    exist_ok=True,
    save_txt=True,              # YOLO 포맷 라벨(txt) 저장
    save_conf=True,             # 각 박스 confidence 저장
    verbose=True
)

print("\n✅ Prediction 완료")
print(f"결과 폴더: {os.path.join(project, 'predict_stage2_1280')}")


In [ ]:
# -----------------------------
# Test 세트 평가 (Evaluation)
#  - stage2(1280) best.pt 기준
# -----------------------------

# Stage2 best.pt 경로 지정
best_stage2 = os.path.join(project, stage2_name, "weights", "best.pt")
assert os.path.exists(best_stage2), f"Stage2 best.pt not found: {best_stage2}"

# 모델 로드
yolo_best_model = YOLO(best_stage2)

# 테스트 세트 평가
metrics = yolo_best_model.val(
    data=data_yaml,             # 동일한 data.yaml 사용
    split="test",               # test 세트 평가 명시
    imgsz=1280,                 # 학습과 동일한 해상도
    batch=8,                    # Colab GPU 여유에 따라 4~8로 조정
    conf=0.25,                  # 평가 기준 confidence threshold
    iou=0.5,                    # mAP50 기준 (COCO mAP50-95는 자동 계산)
    device=0,
    project=project,
    name="test_result_stage2_1280",  # 결과 폴더명
    exist_ok=True,
    plots=True,                 # PR curve, confusion matrix 등 저장
    save_json=True              # COCO 포맷 결과 JSON 저장
)

print("\n✅ Test 세트 평가 완료")
print(metrics)
print(f"결과 폴더: {os.path.join(project, 'test_result_stage2_1280')}")


Epoch: 현재 학습 중인 Epoch 번호

GPU_mem: 현재 GPU 메모리 사용량

box_loss / cls_loss / dfl_loss: 학습 손실(Loss) 값

 box_loss: 바운딩 박스 회귀 손실

 cls_loss: 클래스 분류 손실 (이번 경우엔 leaf=0 하나라서 점점 작아짐)

 dfl_loss: 분포 초점 손실 (YOLO의 anchor-free bounding box 품질)


Instances: 배치에 들어간 객체(바운딩박스) 개수

Size: 입력 이미지 크기 (ex. 640×640)

P (Precision)
→ 탐지한 것 중에서 맞은 비율 (정밀도)

R (Recall)
→ 실제 객체 중에서 탐지한 비율 (재현율)

mAP50 (mean Average Precision at IoU=0.5)
→ 가장 대표적인 정확도 지표.
값이 1.0(100%)이면 거의 완벽한 탐지 성능.

mAP50-95
→ IoU 0.5부터 0.95까지 평균낸 AP. 더 엄격한 지표.
보통 이 값이 모델의 실질적인 성능 평가 기준.

## 2. 분류 모델링(MobileNetV3-Large)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input # MobileNetV3용 입력 정규화 함수

tf.random.set_seed(42)
np.random.seed(42)

# 1. 데이터셋 경로
data_dir = f"/content/{plant_name}_dataset_classification"   # train/ val/ test/ 하위에 normal/ overwatered/ underwatered

# 2. 데이터 전처리 & 증강
# MobileNetV3는 preprocess_input 사용해야 정확도가 높다고 함.
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # 정규화
    rotation_range=15, # 회전
    width_shift_range=0.1, # 이동(width)
    height_shift_range=0.1, # 이동(height)
    zoom_range=0.2, # 확대
    horizontal_flip=True # 좌우 반전
)
val_datagen  = ImageDataGenerator(preprocessing_function=preprocess_input) # 증강 없이 정규화만 진행
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) # 증강 없이 정규화만 진행

# 3. 제너레이터(데이터를 메모리에 한 번에 모두 올리지 않고, 배치 단위로 실시간으로 불러와서 모델에 공급해주는 객체)
BATCH = 32
IMG_SZ = (224, 224)


# 학습 데이터를 디스크에서 배치 단위로 실시간 로딩 + 증강 적용
train_gen = train_datagen.flow_from_directory( # 폴더 구조를 자동으로 라벨링
    directory=f"{data_dir}/train",
    target_size=IMG_SZ,
    batch_size=BATCH,
    class_mode="categorical", # 원핫 인코딩된 벡터로 라벨을 반환
    shuffle=True
)
val_gen = val_datagen.flow_from_directory(
    directory=f"{data_dir}/val",
    target_size=IMG_SZ,
    batch_size=BATCH,
    class_mode="categorical",
    shuffle=False # train 데이터만 셔플해야 함.
)
test_gen = test_datagen.flow_from_directory(
    directory=f"{data_dir}/test",
    target_size=IMG_SZ,
    batch_size=BATCH,
    class_mode="categorical",
    shuffle=False # train 데이터만 셔플해야 함.
)


# 4. 클래스 매핑 확인
# ex.'normal':0, 'overwatered':1, 'underwatered':2 -> cf.이미지 파일명의 0,1,2랑(ex. B-1-H의 1) 관계 없음
print("Train classes:", train_gen.class_indices)
print("Val   classes:", val_gen.class_indices)
print("Test  classes:", test_gen.class_indices)

NUM_CLASSES = train_gen.num_classes # 클래스 개수


# 5. 클래스 불균형 보정
y = train_gen.classes
unique, counts = np.unique(y, return_counts=True) # unique는 각 클래스, 각 클래스 라벨별로 몇 장 있는지 집계(count)
freq = {int(k): int(v) for k, v in zip(unique, counts)} # 딕셔너리로 묶음(ex.{0:800, 1:150, 2:50})
total = len(y) # 전체 데이터 수
class_weight = {c: (total / (NUM_CLASSES * freq[c])) for c in unique} # 전체 샘플수 / (클래스 개수 x 해당 클래스 샘플 수) -> 데이터가 적을수록 weight 커짐
print("class_weight:", class_weight)


# 6. MobileNetV3-Large 모델 로드(ImageNet으로 사전학습된 특징 추출기(분류 헤드 제외) 로드)
# 분류 헤드: 딥러닝 모델 구조에서 마지막 단계(출력부). 딥러닝 모델: 몸통(특징 추출기, backbone) + 머리(분류기, head) 구조
cls_base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(IMG_SZ[0], IMG_SZ[1], 3), # 224×224 크기의 RGB 이미지
    include_top=False, # 기존의 마지막 분류 레이어(softmax 레이어)를 제거하고 가져옴. -> backbone(특징 추출기)만 가져옴
    weights="imagenet" # ImageNet 데이터셋으로 미리 학습된 가중치를 불러옴
)
cls_base_model.trainable = False # 학습 중 이 모델의 가중치를 업데이트하지 않겠다는 의미(사전 학습된 특징 추출기(Backbone)를 그대로 사용)


# 7. Backbone(MobileNetV3Large) 위에 새로운 분류기 헤드 붙이기
# GAP(GlobalAveragePooling2D)으로 특징 맵을 압축 -> 특징 맵의 공간 차원(H×W)을 평균내서 1D 벡터로 바꿈
# 클래스 수만큼 소프트맥스 Dense로 분류기 헤드 구성.
x = layers.GlobalAveragePooling2D()(cls_base_model.output) # cls_base_model.output: MobileNetV3Large에서 나온 특징 맵
x = layers.Dropout(0.4)(x)
output = layers.Dense(NUM_CLASSES, activation="softmax")(x)

# 최종 모델 객체 생성
cls_model = models.Model(inputs=cls_base_model.input, outputs=output)


# 8. 콜백
cb = [
    # 검증 손실이 5 에포크 동안 개선 없으면 중단
    callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),

    # val_loss 기준 베스트 모델만 저장
    callbacks.ModelCheckpoint(f"mobilenetv3_log/val_log/{plant_name}_mobilenetv3_large_best.h5", save_best_only=True, monitor="val_loss"),

    # 정체되면 학습률을 절반으로 낮춰 미세 조정
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

Found 10799 images belonging to 3 classes.
Found 3600 images belonging to 3 classes.
Found 3600 images belonging to 3 classes.
Train classes: {'normal': 0, 'overwatered': 1, 'underwatered': 2}
Val   classes: {'normal': 0, 'overwatered': 1, 'underwatered': 2}
Test  classes: {'normal': 0, 'overwatered': 1, 'underwatered': 2}
class_weight: {np.int32(0): 0.9999074074074074, np.int32(1): 0.9999074074074074, np.int32(2): 1.0001852366398074}
12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# 1st. 분류기 헤드만 학습
cls_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
history = cls_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=class_weight,
    callbacks=cb
)


# 2nd. Fine-tuning (상위 블록 학습)
cls_base_model.trainable = True # Backbone학습을 위해 True로 전환
n_unfreeze = 60  # 마지막 60개 레이어만 학습
for layer in cls_base_model.layers[:-n_unfreeze]:
    layer.trainable = False # 마지막 n_unfreeze개를 제외한 앞부분은 False
# 즉, 저수준 feature (엣지, 색상, 텍스처 등) 은 그대로 두고,
# 고수준 feature (잎 모양, 병충해 패턴 등) 부분을 새 데이터셋에 맞게 미세 조정

cls_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
history_ft = cls_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    class_weight=class_weight,
    callbacks=cb
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
338/338 ━━━━━━━━━━━━━━━━━━━━ 130s 364ms/step - accuracy: 0.6921 - loss: 0.7397 - val_accuracy: 0.9556 - val_loss: 0.1376 - learning_rate: 0.0010
Epoch 2/15
338/338 ━━━━━━━━━━━━━━━━━━━━ 121s 358ms/step - accuracy: 0.9214 - loss: 0.2175 - val_accuracy: 0.9783 - val_loss: 0.0851 - learning_rate: 0.0010
Epoch 3/15
338/338 ━━━━━━━━━━━━━━━━━━━━ 120s 356ms/step - accuracy: 0.9467 - loss: 0.1598 - val_accuracy: 0.9814 - val_loss: 0.0696 - learning_rate: 0.0010
Epoch 4/15
338/338 ━━━━━━━━━━━━━━━━━━━━ 120s 356ms/step - accuracy: 0.9468 - loss: 0.1497 - val_accuracy: 0.9869 - val_loss: 0.0554 - learning_rate: 0.0010
Epoch 5/15
338/338 ━━━━━━━━━━━━━━━━━━━━ 121s 357ms/step - accuracy: 0.9534 - loss: 0.1368 - val_accuracy: 0.9850 - val_loss: 0.0561 - learning_rate: 0.0010
Epoch 6/15
338/338 ━━━━━━━━━━━━━━━━━━━━ 121s 357ms/step - accuracy: 0.9611 - loss: 0.1207 - val_accuracy: 0.9869 - val_loss: 0.0451 - learning_rate: 0.0010
Epoch 7/15
338/338 ━━━━━━━━━━━━━━━━━━━━ 120s 354ms/step - accura

In [ ]:
# 최종 테스트 평가
test_loss, test_acc = cls_model.evaluate(test_gen)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

# 저장
cls_model.save(f"mobilenetv3_log/{plant_name}_mobilenetv3_large_best(final).keras")

113/113 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9990 - loss: 0.0026
Test Accuracy: 0.9986, Test Loss: 0.0038


## 3. 추론(잎 탐지(YOLO) -> 잎 영역 분리 -> 건조/일반/과습 분류)

In [ ]:
import os, glob, csv, random
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from ultralytics import YOLO

# 경로/설정
DET_WEIGHTS = f"yolo_log/{plant_name}_yolo/detect/train/weights/best.pt"   # YOLO 잎 탐지 모델
CLS_WEIGHTS = f"mobilenetv3_log/{plant_name}_mobilenetv3_large_best(final).h5"       # MobileNetV3-Large 분류 모델
SOURCE = f"/content/{plant_name}_dataset_detection/test/images"       # 추론할 이미지들

OUT_DIR = f"yolo_log/{plant_name}_yolo/detect/test_results/{plant_name}_test_result_outputs" # 테스트 결과 저장 폴더
DRAW_DIR = os.path.join(OUT_DIR, "draws") # 시각화한 이미지(바운딩 박스 포함)
CROP_DIR = os.path.join(OUT_DIR, "crops") # 탐지된 잎 부분만 잘라낸 이미지
os.makedirs(DRAW_DIR, exist_ok=True) # 폴더가 이미 있으면 에러를 내지 않고 넘어감
os.makedirs(CROP_DIR, exist_ok=True)

YOLO_CONF = 0.5
YOLO_IOU  = 0.45
IMG_SIZE  = 640
CLS_SIZE  = (224, 224)

CLASS_NAMES = ["normal", "overwatered", "underwatered"] # 알파벳 순(디렉터리에 이렇게 정렬되어 있음)

# 모델 로드
detection_model = YOLO(DET_WEIGHTS)
clssification_model = tf.keras.models.load_model(CLS_WEIGHTS, compile=False)

# 유틸 함수
def list_images(path):
    if os.path.isdir(path):
        imgs = glob.glob(os.path.join(path, "*.jpg"))
        return sorted(imgs)
    return [path]

def preprocess_for_cls(bgr_img, size=(224,224)):
    rgb = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
    rgb = cv2.resize(rgb, size, interpolation=cv2.INTER_AREA)
    x = preprocess_input(rgb.astype(np.float32))
    return np.expand_dims(x, axis=0)

def clip_box(x1, y1, x2, y2, W, H):
    x1 = max(0, min(int(x1), W-1))
    y1 = max(0, min(int(y1), H-1))
    x2 = max(0, min(int(x2), W-1))
    y2 = max(0, min(int(y2), H-1))
    return x1, y1, x2, y2

def get_true_label_from_filename(img_name):
    """
    파일명 패턴에서 GT 클래스 추출
    규칙:
      'B-1-?' 또는 'L-1-?' → underwatered
      'B-2-?' 또는 'L-2-?' → normal
      'B-3-?' 또는 'L-3-?' → overwatered
    """
    tokens = img_name.split("-")
    for i in range(1, len(tokens) - 1):
        if tokens[i] in {"1","2","3"} and tokens[i-1].isalpha() and tokens[i+1].isalpha():
            return {"1":"underwatered", "2":"normal", "3":"overwatered"}[tokens[i]]
    raise ValueError(f"GT 클래스를 추출할 수 없음: {img_name}")

# 입력 이미지 준비
image_list = list_images(SOURCE)
assert len(image_list) > 0, f"No images found in {SOURCE}"

# 추론 루프
csv_path = os.path.join(OUT_DIR, "test_result_summary.csv")
with open(csv_path, "w", newline="", encoding="utf-8") as csv_f:
    csv_w = csv.writer(csv_f)
    csv_w.writerow(["image","leaf_idx","x1","y1","x2","y2","det_conf","pred_class","pred_prob","true_class","correct"])

    for img_path in image_list:
        img_name = os.path.basename(img_path)
        draw = cv2.imread(img_path)
        if draw is None:
            print(f"[WARN] Cannot read image: {img_path}")
            continue
        H, W = draw.shape[:2]

        true_label = get_true_label_from_filename(img_name)

        # YOLO 탐지
        results = detection_model.predict(source=img_path, imgsz=IMG_SIZE,
                                    conf=YOLO_CONF, iou=YOLO_IOU, verbose=False)
        r = results[0]; boxes = r.boxes

        if boxes is None or len(boxes) == 0:
            cv2.imwrite(os.path.join(DRAW_DIR, img_name), draw)
            print(f"[INFO] No leaves detected: {img_name}")
            continue

        # 박스별 분류
        for i, b in enumerate(boxes):
            x1, y1, x2, y2 = b.xyxy[0].tolist()
            conf = float(b.conf[0])
            x1, y1, x2, y2 = clip_box(x1, y1, x2, y2, W, H)

            leaf = draw[y1:y2, x1:x2].copy()
            if leaf.size == 0: continue

            x = preprocess_for_cls(leaf, CLS_SIZE)
            probs = clssification_model.predict(x, verbose=0)[0]
            cls_id = int(np.argmax(probs))
            cls_name = CLASS_NAMES[cls_id]
            cls_prob = float(probs[cls_id])

            correct = int(cls_name == true_label)

            # 시각화
            label = f"{cls_name} {cls_prob:.2f} | det {conf:.2f}"
            cv2.rectangle(draw, (x1, y1), (x2, y2), (0,200,0), 2)
            cv2.rectangle(draw, (x1, y1-22), (x1 + max(80, 8*len(label)), y1), (0,200,0), -1)
            cv2.putText(draw, label, (x1+4, y1-6), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0,0,0), 1, cv2.LINE_AA)

            cv2.imwrite(os.path.join(CROP_DIR, f"{os.path.splitext(img_name)[0]}_leaf{i}_{cls_name}.jpg"), leaf)
            csv_w.writerow([img_name,i,x1,y1,x2,y2,f"{conf:.4f}",cls_name,f"{cls_prob:.4f}",true_label,correct])

        cv2.imwrite(os.path.join(DRAW_DIR, img_name), draw)
        #print(f"[DONE] {img_name} → {os.path.join(DRAW_DIR, img_name)}")

# 정확도 계산
df = pd.read_csv(csv_path)
acc = df["correct"].mean() if len(df) else 0.0
print(f"최종 정확도: {acc:.4f} (총 샘플={len(df)})")
print(f"결과 CSV: {csv_path}")
print(f"시각화 이미지: {DRAW_DIR}")
print(f"crop 이미지: {CROP_DIR}")

KeyboardInterrupt: 